In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [2]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [3]:
def splitting_def(df):
    date_range_1 = (df.index >= '2020-05-01') & (df.index <= '2020-06-25')
    date_range_2 = (df.index >= '2023-05-01') & (df.index <= '2023-06-15')

    # Combine the date ranges to create the test set
    test_set = df[date_range_1 | date_range_2]

    # The rest of the data will be your training set
    training_set = df[~(date_range_1 | date_range_2)]
    
    # Splitting the test_set into X_test and y_test
    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement']  # Assuming 'pv_measurement' is your target variable

    # Splitting the training_set into X_train and y_train
    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']
    
    return X_train, X_test, y_train, y_test

X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = splitting_def(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))

In [4]:
#Create a pool of data
train_pool_b = Pool(X_train_new_b, y_train_new_b)

test_pool_b = Pool(X_test_new_b) 

In [6]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "has_time": trial.suggest_categorical('has-time', [True, False]),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "random_state": 2,
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "nan_mode": trial.suggest_categorical("nan_mode", ["Min", "Max"])
    }

    catboost_model_b = CatBoostRegressor(**params, verbose=100)
    catboost_model_b.fit(train_pool_b)
    pred_b = pd.DataFrame(catboost_model_b.predict(test_pool_b))
    MAE_a = mean_absolute_error(y_test_b, pred_b)
    return MAE_a

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=60)

[I 2023-11-06 20:06:55,061] A new study created in memory with name: no-name-61cd20d6-8e5f-4c1d-9c96-75f60a58776d


0:	learn: 181.7833532	total: 20.2ms	remaining: 27.7s
100:	learn: 86.6260792	total: 1.33s	remaining: 16.8s
200:	learn: 80.2806198	total: 2.65s	remaining: 15.4s
300:	learn: 77.4019209	total: 3.97s	remaining: 14.1s
400:	learn: 75.1395839	total: 5.29s	remaining: 12.8s
500:	learn: 73.3085371	total: 6.67s	remaining: 11.6s
600:	learn: 71.6058550	total: 8.25s	remaining: 10.6s
700:	learn: 70.0338949	total: 9.92s	remaining: 9.51s
800:	learn: 68.6558287	total: 12.3s	remaining: 8.75s
900:	learn: 67.3792694	total: 14.9s	remaining: 7.79s
1000:	learn: 66.1118030	total: 17.6s	remaining: 6.52s
1100:	learn: 65.0415381	total: 20.1s	remaining: 4.96s
1200:	learn: 64.0522875	total: 22.7s	remaining: 3.24s
1300:	learn: 63.0120884	total: 25.5s	remaining: 1.41s


[I 2023-11-06 20:07:22,804] Trial 0 finished with value: 81.71095025797248 and parameters: {'iterations': 1373, 'learning_rate': 0.029662339445065907, 'depth': 6, 'min_data_in_leaf': 33, 'l2_leaf_reg': 3, 'has-time': True, 'bagging_temperature': 0.7771074112218899, 'random_strength': 0.3972311167911148, 'border_count': 980, 'rsm': 0.10510525024422139, 'nan_mode': 'Max'}. Best is trial 0 with value: 81.71095025797248.


1372:	learn: 62.3817487	total: 27.4s	remaining: 0us
0:	learn: 184.0788143	total: 10ms	remaining: 16.5s
100:	learn: 125.7080104	total: 651ms	remaining: 9.96s
200:	learn: 105.5223418	total: 1.28s	remaining: 9.24s
300:	learn: 98.4883823	total: 1.97s	remaining: 8.81s
400:	learn: 95.2729027	total: 2.67s	remaining: 8.31s
500:	learn: 93.4794487	total: 3.33s	remaining: 7.61s
600:	learn: 92.1111818	total: 4.14s	remaining: 7.2s
700:	learn: 91.0976001	total: 4.87s	remaining: 6.57s
800:	learn: 90.2491929	total: 5.65s	remaining: 5.96s
900:	learn: 89.5150722	total: 6.31s	remaining: 5.22s
1000:	learn: 88.9180836	total: 6.85s	remaining: 4.42s
1100:	learn: 88.3892664	total: 7.55s	remaining: 3.74s
1200:	learn: 87.8900579	total: 8.19s	remaining: 3.04s
1300:	learn: 87.4543673	total: 8.77s	remaining: 2.33s
1400:	learn: 87.0270368	total: 9.38s	remaining: 1.65s
1500:	learn: 86.6168730	total: 10.1s	remaining: 983ms
1600:	learn: 86.2294817	total: 10.7s	remaining: 308ms


[I 2023-11-06 20:07:34,248] Trial 1 finished with value: 73.41767324211861 and parameters: {'iterations': 1647, 'learning_rate': 0.008285667949530987, 'depth': 2, 'min_data_in_leaf': 7, 'l2_leaf_reg': 6, 'has-time': True, 'bagging_temperature': 0.638344903820208, 'random_strength': 0.9388444992830671, 'border_count': 434, 'rsm': 0.8347735297026142, 'nan_mode': 'Max'}. Best is trial 1 with value: 73.41767324211861.


...

2300:	learn: 84.4164076	total: 19.5s	remaining: 110ms
2313:	learn: 84.3736044	total: 19.6s	remaining: 0us
0:	learn: 184.6028908	total: 4.2ms	remaining: 5.37s
100:	learn: 152.2601989	total: 925ms	remaining: 10.8s
200:	learn: 132.0109163	total: 1.59s	remaining: 8.56s
300:	learn: 119.4848834	total: 2.41s	remaining: 7.84s
400:	learn: 111.7194402	total: 3.31s	remaining: 7.26s
500:	learn: 106.8513932	total: 4.39s	remaining: 6.83s
600:	learn: 103.6569606	total: 5.7s	remaining: 6.45s
700:	learn: 101.4240689	total: 6.62s	remaining: 5.47s
800:	learn: 99.8393631	total: 7.47s	remaining: 4.47s
900:	learn: 98.6988566	total: 8.41s	remaining: 3.54s
1000:	learn: 97.8732580	total: 9.59s	remaining: 2.68s
1100:	learn: 97.2144539	total: 10.5s	remaining: 1.72s
1200:	learn: 96.6905123	total: 11.6s	remaining: 776ms


[I 2023-11-06 21:26:44,335] Trial 58 finished with value: 96.48628848103215 and parameters: {'iterations': 1281, 'learning_rate': 0.003350755375908208, 'depth': 3, 'min_data_in_leaf': 13, 'l2_leaf_reg': 5, 'has-time': True, 'bagging_temperature': 0.6681617954585722, 'random_strength': 0.8227413591551932, 'border_count': 4, 'rsm': 0.7738962917491201, 'nan_mode': 'Min'}. Best is trial 36 with value: 73.11068914175323.


1280:	learn: 96.3155597	total: 12.4s	remaining: 0us
0:	learn: 184.0588026	total: 6.62ms	remaining: 12.5s
100:	learn: 127.9024154	total: 579ms	remaining: 10.3s
200:	learn: 108.2130411	total: 1.45s	remaining: 12.2s
300:	learn: 101.0585783	total: 2.03s	remaining: 10.7s
400:	learn: 98.0998540	total: 2.69s	remaining: 10s
500:	learn: 96.6047847	total: 3.27s	remaining: 9.08s
600:	learn: 95.6892196	total: 4.01s	remaining: 8.63s
700:	learn: 95.0566554	total: 4.59s	remaining: 7.8s
800:	learn: 94.5927640	total: 5.38s	remaining: 7.34s
900:	learn: 94.2470583	total: 6.09s	remaining: 6.7s
1000:	learn: 93.9753006	total: 6.72s	remaining: 5.99s
1100:	learn: 93.7472652	total: 7.64s	remaining: 5.49s
1200:	learn: 93.5525905	total: 8.16s	remaining: 4.7s
1300:	learn: 93.3820449	total: 8.87s	remaining: 4.03s
1400:	learn: 93.2256376	total: 9.38s	remaining: 3.29s
1500:	learn: 93.0813980	total: 10.1s	remaining: 2.64s
1600:	learn: 92.9480646	total: 11.1s	remaining: 2.02s
1700:	learn: 92.8240957	total: 11.8s	remai

[I 2023-11-06 21:26:58,002] Trial 59 finished with value: 78.92093565373548 and parameters: {'iterations': 1893, 'learning_rate': 0.010191181240586593, 'depth': 1, 'min_data_in_leaf': 52, 'l2_leaf_reg': 7, 'has-time': True, 'bagging_temperature': 0.563915315463397, 'random_strength': 0.701782233977871, 'border_count': 439, 'rsm': 0.9610229820913956, 'nan_mode': 'Max'}. Best is trial 36 with value: 73.11068914175323.


1892:	learn: 92.6082742	total: 13.1s	remaining: 0us


In [9]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)

with open("optuna-best-parameters_b.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 1480, 'learning_rate': 0.004154859937303772, 'depth': 1, 'colsample_bylevel': 0.8980589862889544, 'min_data_in_leaf': 11, 'l2_leaf_reg': 7, 'has-time': True, 'bagging_temperature': 0.8588447919947089, 'random_strength': 0.5000970078140154, 'border_count': 358, 'rsm': 0.31318686704086646, 'nan_mode': 'min'}
82.32735192987839
